# i18

The i18n library provides you an easier way to manage languages with grammars and lexers.

## Replace a string given a grammar and a replace function


```python
sub(grammar, fn_replace, string)
```

In Lark syntax, where the grammar expects a group symbol, fn_replace is a function that replaces this group symbol; any other symbols remain unchanged. The string adheres to the rules of a Context-Free Grammar (CFG) rather than a regular expression (regex). 

In [80]:
from i18.parser import sub

sub("""
start: expression

expression: atom ("+"|"-"|"*"|"/") expression | atom

atom: SIGNED_NUMBER -> group
    | "(" expression ")"

%import common.SIGNED_NUMBER
%import common.WS
%ignore WS

""", lambda token: str(int(token)%2), "(4+(5+6)+5)+3")

'(0+(1+0)+1)+1'

## Translate your blade templates

When you have embedded code in HTML, you can translate both without losing any information

In [32]:
from i18.grammars import html_grammar
from i18.parser import sub
from deep_translator import (GoogleTranslator,DeeplTranslator)

translator = GoogleTranslator(source='auto', target='de')

new_text = sub(html_grammar, lambda token: translator.translate(text=token), """
<span class="flex">
     @if($step == 1)
            <span class="text-blue-600">Hello <p>World {{$x->g}}</p></span>
     @else
            <span class="text-green-500">You're welcome</span>
     @endif
</span>
""")
print(new_text)


<span class="flex">
     @if($step == 1)
            <span class="text-blue-600">Hallo<p>Welt {{$x->g}}</p></span>
     @else
            <span class="text-green-500">Gern geschehen</span>
     @endif
</span>



In [6]:
from i18.grammars import html_grammar
from i18.parser import sub
from deep_translator import (GoogleTranslator,DeeplTranslator)

translator = GoogleTranslator(source='auto', target='de')

new_text = sub(html_grammar, lambda token: "0", """<p>{{$x->g}}</p>""")
print(new_text)

<p>0</p>


# Search in string based on Grammar

In [28]:
from i18.parser import search
from i18.grammars import json_grammar, sub_in_grammar

search(sub_in_grammar(json_grammar, {"GROUP": r"/[\w_.]+@[\w_.]+\.[\w-]{2,4}/"}), 
       '{"key": ["abdiel@apimarket.mx"], "abdiel2@apimarket.mx": "2"}')

[Token('__ANON_1', 'abdiel@apimarket.mx')]

# Lexer

A lexer converts a sequence of characters into a sequence of tokens, but these tokens have a meaning associated with them. This task is quite similar to named-entity recognition (yes, you could use Spacy), however, a classic lexer does the job.

In [4]:
from i18.lexer import Lexer
lexer = Lexer()
[token for token in lexer.tokenize("""{{!!$person->name!!}}""")]

[LexToken(BLADE_ECHO,'{{!!$person->name!!}}',1,0)]

# Apply i18

In [3]:
from i18 import apply_i18

new_text, translations = apply_i18("""
 <div class="ml-3 relative">
        Hello World 
        <p>How are you? {{$x->g()}}</p>
  </div>
""")
print(new_text)


 <div class="ml-3 relative">
        {{__("hello_world")}}<p>{{__("how_are_you?")}} {{$x->g()}}</p>
  </div>



In [2]:
translations

{'es': {'hello_world': 'Hola Mundo', 'how_are_you?': '¿Cómo estás?'},
 'en': {'hello_world': 'Hello World', 'how_are_you?': 'How are you?'}}